In [2]:
import torch
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms


In [13]:
# Create tensors.
x = torch.tensor(4., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

In [14]:
y = w * x + b  
y

tensor(11., grad_fn=<AddBackward0>)

In [15]:
# Compute gradients.
y.backward()

In [16]:
# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2.)
tensor(4.)
tensor(1.)


## 2. Basic autograd example 2                     

In [17]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

In [37]:
y

tensor([[1, 2],
        [3, 4]], dtype=torch.int32)

In [19]:
# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

w:  Parameter containing:
tensor([[-0.1654, -0.5098,  0.4362],
        [-0.2970, -0.1043, -0.5728]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.3493, -0.1000], requires_grad=True)


In [39]:
linear.parameters

<bound method Module.parameters of Linear(in_features=3, out_features=2, bias=True)>

In [21]:
#help(linear)

In [22]:
# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)


In [41]:
pred = linear(x)


AttributeError: 'numpy.ndarray' object has no attribute 'dim'

In [24]:
# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

loss:  1.097792625427246


In [25]:

# Backward pass.
loss.backward()

In [26]:
# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

dL/dw:  tensor([[-0.5779, -1.0459,  0.2119],
        [-0.3150, -0.1595, -0.2922]])
dL/db:  tensor([ 0.3790, -0.1807])


In [27]:
optimizer.step()

In [28]:
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss after 1 step optimization:  1.0793378353118896


## 3. Loading data from numpy     

In [29]:
x = np.array([[1, 2], [3, 4]])


In [30]:
# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

In [31]:
y

tensor([[1, 2],
        [3, 4]], dtype=torch.int32)

In [33]:
z = y.numpy()
z

array([[1, 2],
       [3, 4]])

##  4. Input pipeline   

In [34]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

100.0%

Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [42]:
# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

torch.Size([3, 32, 32])
6


In [43]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [44]:
# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

In [45]:
# Mini-batch images and labels.
images, labels = data_iter.next()

In [46]:
# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

## 5. Input pipeline for custom dataset       

In [47]:
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)

ValueError: num_samples should be a positive integer value, but got num_samples=0

##  6. Pretrained model      

In [36]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to C:\Users\5001/.cache\torch\hub\checkpoints\resnet18-5c106cde.pth
100.0%


In [48]:
for param in resnet.parameters():
    param.requires_grad = False

In [49]:
# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.


In [50]:
# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)


torch.Size([64, 100])


## 7. Save and load the model  

In [51]:
# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

<All keys matched successfully>